# Implement your own energy evaluator interface

This notebook tutorial shows the basic steps to implement your own interface for your preferred energy evaluation method. CARE implements GAME-Net-UQ as default estimator for the intermediate and reaction properties, but no worries, if you don't like it or you prefer other methods, here we show the basic steps to do so.

Step 1: Import the base classes for intermediates and reactions. You will have to define a specific interface for your own model as child classes of the one imported here.

In [ ]:
from care.evaluators import IntermediateEnergyEstimator, ReactionEnergyEstimator
from care.evaluators.gamenet_uq import GameNetUQInter, GameNetUQRxn

Check the method resolution order (*mro*) for the interfaces implementing GAME-Net-UQ, you will see that these are defined as subclasses of ``IntermediateEnergyEstimator``  and ``ReactionEnergyEstimator``.

In [ ]:
GameNetUQInter.__mro__  

In [ ]:
GameNetUQRxn.__mro__

## Intermediate evaluator interface

We will implement here a model based on group additivity with random values. The main effort involves implementing the method ``IntermediateEnergyEstimator.eval()``, which thakes as input the ``Intermediate`` instance and returns it with the correponsing energy evalautino as instance attribute.

In [ ]:
from care import Intermediate, Surface
import random

class ExampleIntermediate(IntermediateEnergyEstimator):
    def __init__(
        self, surface: Surface
    ):
        """Example Interface
        """

        self.surface = surface
        self.metal_contribution = random.random()  # RANDOM VALUES FOR TUTORIAL PURPOSES!!!

    def adsorbate_domain(self):
        return ['C', 'H', 'O', 'N', 'S']

    def surface_domain(self):
        return ['Pd']        

    def eval(
        self,
        intermediate: Intermediate,
    ) -> None:

        if intermediate.phase == "surf":  # active site
            intermediate.energy = 0.0
        elif intermediate.phase == "gas":  # gas phase
            intermediate.energy = intermediate.ref_energy()
        elif intermediate.phase == "ads":  # adsorbed
            intermediate.energy = intermediate.ref_energy() + self.metal_contribution
        else:
            raise ValueError("Phase not supported by the current estimator.")

        return intermediate

## Reaction evaluator interface

Important note: The reaction properties evaluator interface could depend on the way you implement the intermediate energy interface!

In [ ]:
from care import ElementaryReaction

class ExampleReaction(ReactionEnergyEstimator):
    """
    Example of Interface for evaluating reaction properties.
    """

    def __init__(
        self,
        intermediates: dict[str, Intermediate],
    ):
        self.intermediates = intermediates

    def adsorbate_domain(self):
        return ['C', 'H', 'O', 'N', 'S']

    def surface_domain(self):
        return ['Pd']  

    def calc_reaction_energy(self, reaction: ElementaryReaction) -> None:
        """
        Get the reaction energy of the elementary reaction.

        Args:
            reaction (ElementaryReaction): Elementary reaction.
        """
        e_is, e_fs = 0.0, 0.0
        
        for reactant in reaction.reactants:
            if reactant.is_surface:
                continue
            e_is += abs(reaction.stoic[reactant.code]) * self.intermediates[reactant.code].energy  # energy attribute is set by the IntermediateEnergyEstimator
        for product in reaction.products:
            if product.is_surface:
                continue
            e_fs += abs(reaction.stoic[product.code]) * self.intermediates[product.code].energy  # energy attribute is set by the IntermediateEnergyEstimator

        reaction.e_is = e_is
        reaction.e_fs = e_fs        
        reaction.e_rxn = e_fs - e_is

    def calc_reaction_barrier(self, reaction: ElementaryReaction) -> None:
        """
        Get BEP activation energy of the elementary reaction.

        Args:
            reaction (ElementaryReaction): Elementary reaction.
        """
        alpha = 0.5  # scaling factor for activation energy (RANDOM VALUES!!!)
        beta = 0.25 # scaling factor for reaction energy (RANDOM VALUES!!!)
        reaction.e_ts = alpha * reaction.e_fs + (1 - alpha) * reaction.e_is + beta        
        reaction.e_act = reaction.e_ts - reaction.e_is

    def eval(
        self,
        reaction: ElementaryReaction,
    ) -> None:
        """
        Estimate the reaction energy properties.

        Args:
            reaction (ElementaryReaction): The elementary reaction.
        """
        self.calc_reaction_energy(reaction)
        self.calc_reaction_barrier(reaction)
        return reaction

# Generate CRN with your own interfaces!

In [ ]:
from care import gen_blueprint
from care.evaluators.gamenet_uq import load_surface

In [ ]:
inters, rxns = gen_blueprint(2, 1, False, False, False)  # generate a blueprint with 2 intermediates and 1 reaction

In [ ]:
surface = load_surface('Pt', '110')  # load a surface from the database

In [ ]:
your_intermediate_estimator = ExampleIntermediate(surface)

In [ ]:
inters_evaluated = {}
for k, inter in inters.items():
    
    inters_evaluated[k] = your_intermediate_estimator.eval(inter)
    print(inter, '    ', inters_evaluated[k].energy)

In [ ]:
your_reaction_estimator = ExampleReaction(inters_evaluated)

In [ ]:
rxns_evaluated = []
for rxn in rxns:
    rxns_evaluated.append(your_reaction_estimator.eval(rxn))
    print(rxn.repr_hr, '    ', rxns_evaluated[-1].e_rxn, '    ', rxns_evaluated[-1].e_act)

rxns_evaluated = sorted(rxns_evaluated)

Observations:
- **The values shown here are random**, and the formulas used here are extremely simple, but this is just to show how to implement the interface classes. You are free to implement whatever model you prefer (e.g., GNNs, MLPs, group-additivity based, etc.)
- The examples shown here do not take into account the dependence on variables like temperature, pressure, pH, etc. but you can easily include them as class attributes
- Contact us if you need support in implementing more complex evalautors! :)